In [1]:
!pip install keras-tuner

# California Housing Dataset

In this example, we are using the default dataset California Housing Dataset present in the Google colab.

## Importing Libraries

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn

  Using cached scikit_learn-1.0.2-cp37-cp37m-win_amd64.whl (7.1 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install keras==2.6

Note: you may need to restart the kernel to use updated packages.


In [11]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import kerastuner as kt

C:\Users\Munna\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if sys.path[0] == "":


In [ ]:
pip --version

In [16]:
train_df = pd.read_csv(r"https://raw.githubusercontent.com/vtech20/keras_tuner_sample/main/california_housing_train.csv",sep=",")
test_df = pd.read_csv(r"https://raw.githubusercontent.com/vtech20/keras_tuner_sample/main/california_housing_test.csv",sep=",")

In [17]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [18]:
test_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [19]:
#shape of train and test
print(train_df.shape)
print(test_df.shape)

(17000, 9)
(3000, 9)


## Dataset Description

In [20]:
train_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [21]:
test_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.00000
mean,-119.589200,35.63539,28.845333,2599.578667,529.950667,1402.798667,489.91200,3.807272,205846.27500
std,1.994936,2.12967,12.555396,2155.593332,415.654368,1030.543012,365.42271,1.854512,113119.68747
min,-124.180000,32.56000,1.000000,6.000000,2.000000,5.000000,2.00000,0.499900,22500.00000
25%,-121.810000,33.93000,18.000000,1401.000000,291.000000,780.000000,273.00000,2.544000,121200.00000
50%,-118.485000,34.27000,29.000000,2106.000000,437.000000,1155.000000,409.50000,3.487150,177650.00000
75%,-118.020000,37.69000,37.000000,3129.000000,636.000000,1742.750000,597.25000,4.656475,263975.00000
max,-114.490000,41.92000,52.000000,30450.000000,5419.000000,11935.000000,4930.00000,15.000100,500001.00000


The distribution of the train and test are pretty same

## Null value check

In [23]:
train_df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [24]:
test_df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

## Split data into the dependent and independent features

In [25]:
TARGET_FEATURE = 'median_house_value'

In [26]:
x_train,y_train = train_df.drop(TARGET_FEATURE,axis=1), train_df[[TARGET_FEATURE]]
x_test,y_test = test_df.drop(TARGET_FEATURE,axis=1),test_df[[TARGET_FEATURE]]

## Feature Scaling

In [27]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Keras Tuner

An open-source Python library called Keras Tuner was created specifically for adjusting the hyperparameters of artificial neural networks. Currently, the Keras tuner supports four different types of tuners or algorithms:

   + Bayesian Optimization
   + Hyperband
   + Sklearn
   + Random Search
The Hyper model is the one you want to tune.

   + Using the model-building feature we can enable this tuning feature

We are going to build the ann with 4 layers

   1. Input layer
   2. Hidden layer 1
   3. Hidden layer 2
   4. Output Layer
unit1, unit2, unit3 will have the range between 25 to 512 with step 32

These will be apply to input layer and hidden layers

The learning rate will be picked up based on given inputs and hence we are use choice method to perform that action

Since this is the regression problem, we are using 'linear' activation function in the output layer. on the hidden layers we are using 'relu'.

For weight updation, we are using adam optimizers and the loss will be MSLE (Mean Squarred Log Error)

In [28]:
msle = MeanSquaredLogarithmicError()

def build_annmodel(hp):
    model = tf.keras.Sequential()
    
    units1 = hp.Int('units1',min_value=25,max_value=512, step=32)
    units2 = hp.Int('units2',min_value=25,max_value=512, step=32)
    units3 = hp.Int('units3',min_value=25,max_value=512, step=32)
    model.add(Dense(units=units1,activation='relu'))
    model.add(tf.keras.layers.Dense(units=units2,activation='relu'))
    model.add(tf.keras.layers.Dense(units=units3,activation='relu'))
    model.add(Dense(1,kernel_initializer='normal',activation='linear'))
    
    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])
    
    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
    loss=msle,
    metrics=[msle]
    )
    
    return model
    
    

 
In order to get a high-performing model quickly, the Hyperband tuning algorithm makes use of adaptive resource allocation and early-stopping.

In [29]:
# HyperBand algorithm from keras tuner

tuner = kt.Hyperband(
    build_annmodel,
    objective="val_mean_squared_logarithmic_error",
    max_epochs=10,
    directory = 'keras_tuner_dir',
    project_name = 'keras_tuner_demo'
        )

In [30]:
tuner.search(x_train,y_train,epochs=10,validation_split = 0.2)

Trial 30 Complete [00h 00m 18s]
val_mean_squared_logarithmic_error: 6.896678447723389

Best val_mean_squared_logarithmic_error So Far: 0.1084776446223259
Total elapsed time: 00h 04m 32s
INFO:tensorflow:Oracle triggered exit


## Best hyper paremeters

In [31]:
for p in ['units1','units2','units3','learning_rate']:
    print(p,tuner.get_best_hyperparameters()[0].get(p))

units1 473
units2 217
units3 217
learning_rate 0.01


## Model Evaluation

In [32]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [35]:
model = tuner.hypermodel.build(best_hps)

In [36]:
history = model.fit(x_train,y_train,epochs=50,validation_split=0.2 )

Epoch 1/50
425/425 [==============================] - 2s 3ms/step - loss: 2.6821 - mean_squared_logarithmic_error: 2.6821 - val_loss: 0.3854 - val_mean_squared_logarithmic_error: 0.3866
Epoch 2/50
425/425 [==============================] - 1s 3ms/step - loss: 0.1620 - mean_squared_logarithmic_error: 0.1620 - val_loss: 0.1881 - val_mean_squared_logarithmic_error: 0.1916
Epoch 3/50
425/425 [==============================] - 1s 2ms/step - loss: 0.1120 - mean_squared_logarithmic_error: 0.1120 - val_loss: 0.1328 - val_mean_squared_logarithmic_error: 0.1354
Epoch 4/50
425/425 [==============================] - 1s 3ms/step - loss: 0.0987 - mean_squared_logarithmic_error: 0.0987 - val_loss: 0.1246 - val_mean_squared_logarithmic_error: 0.1274
Epoch 5/50
425/425 [==============================] - 1s 3ms/step - loss: 0.0942 - mean_squared_logarithmic_error: 0.0942 - val_loss: 0.1218 - val_mean_squared_logarithmic_error: 0.1246
Epoch 6/50
425/425 [==============================] - 2s 5ms/step - lo

Epoch 45/50
425/425 [==============================] - 2s 5ms/step - loss: 0.0600 - mean_squared_logarithmic_error: 0.0600 - val_loss: 0.1080 - val_mean_squared_logarithmic_error: 0.1113
Epoch 46/50
425/425 [==============================] - 2s 5ms/step - loss: 0.0591 - mean_squared_logarithmic_error: 0.0591 - val_loss: 0.1051 - val_mean_squared_logarithmic_error: 0.1083
Epoch 47/50
425/425 [==============================] - 1s 3ms/step - loss: 0.0589 - mean_squared_logarithmic_error: 0.0589 - val_loss: 0.1123 - val_mean_squared_logarithmic_error: 0.1154
Epoch 48/50
425/425 [==============================] - 2s 5ms/step - loss: 0.0598 - mean_squared_logarithmic_error: 0.0598 - val_loss: 0.1197 - val_mean_squared_logarithmic_error: 0.1235
Epoch 49/50
425/425 [==============================] - 2s 5ms/step - loss: 0.0586 - mean_squared_logarithmic_error: 0.0586 - val_loss: 0.1007 - val_mean_squared_logarithmic_error: 0.1035
Epoch 50/50
425/425 [==============================] - 1s 2ms/ste